# Statistics 

In [45]:
from bs4 import BeautifulSoup,SoupStrainer
import requests
import pandas as pd 
import numpy as np
import sqlalchemy
import time
import random 
import tqdm

In [59]:
engine = sqlalchemy.create_engine('sqlite:///gygia')

In [22]:
frame = pd.read_sql('''SELECT * FROM gygia''',engine)

In [23]:
frame.head()

,name,link
0,Ivan Maruschak,https://www.mql5.com/en/signals/1679839?source...
1,kukhan kim,https://www.mql5.com/en/signals/1736795?source...
2,Roman Kartezhnikov,https://www.mql5.com/en/signals/683917?source=...
3,KAUSER AHMED,https://www.mql5.com/en/signals/1784201?source...
4,Marat Timirgazin,https://www.mql5.com/en/signals/1249249?source...


In [24]:
keep_scrap = list(frame.name)

In [51]:
df1 = pd.DataFrame([])
    
for rank,name in enumerate(keep_scrap):
    try : 
    
        dico = {}
        # configure url 
        time.sleep(random.randrange(1,5))
        url = frame.loc[frame.name == name,'link'].values[0]
        print(url)
        response = requests.get(url)
        season_page = BeautifulSoup(response.content)

        # load stat onglet 
        globale = season_page.find('div',attrs={'id' :"tab_content_stats"})

        scroller = globale.findAll('div',attrs={'class' :"s-data-columns__item"})

        map_label_val = {}

        map_label_desc = {}

        for i in scroller:

            stat_description = i['title']

            label = i.find('div',attrs={'class' :"s-data-columns__label"}).text

            value = i.find('div',attrs={'class' :"s-data-columns__value"}).text

            map_label_val[label]=value

            map_label_desc[label]=stat_description


        df2 = pd.DataFrame([map_label_val])
        df2.loc[0,'name'] = name
        df2.loc[0,'last_refresh'] = pd.to_datetime('now')

        df1 = pd.concat([df1,df2])

        print("="*50)
        print(f'Itération -- {rank}')
        print(f'{rank - len(keep_scrap)}')
        print("="*50)

    except : 
        pass
    
    

https://www.mql5.com/en/signals/1679839?source=Site+Signals+MT4+Tile+All
Itération -- 0
-977
https://www.mql5.com/en/signals/1736795?source=Site+Signals+MT4+Tile+All
Itération -- 1
-976
https://www.mql5.com/en/signals/683917?source=Site+Signals+MT4+Tile+All
Itération -- 2
-975
https://www.mql5.com/en/signals/1784201?source=Site+Signals+MT4+Tile+All
Itération -- 3
-974
https://www.mql5.com/en/signals/1249249?source=Site+Signals+MT4+Tile+All
Itération -- 4
-973
https://www.mql5.com/en/signals/1752670?source=Site+Signals+MT4+Tile+All
Itération -- 5
-972
https://www.mql5.com/en/signals/597211?source=Site+Signals+MT4+Tile+All
Itération -- 6
-971
https://www.mql5.com/en/signals/1371953?source=Site+Signals+MT4+Tile+All
Itération -- 7
-970
https://www.mql5.com/en/signals/807415?source=Site+Signals+MT4+Tile+All
Itération -- 8
-969
https://www.mql5.com/en/signals/1575241?source=Site+Signals+MT4+Tile+All
Itération -- 9
-968
https://www.mql5.com/en/signals/723505?source=Site+Signals+MT4+Tile+All
I

In [54]:
df1.isna().sum()

Trades:                         0
Profit Trades:                  0
Loss Trades:                    0
Best trade:                     0
Worst trade:                    0
Gross Profit:                   0
Gross Loss:                     0
Maximum consecutive wins:       0
Maximal consecutive profit:     0
Sharpe Ratio:                   0
Trading activity:               0
Max deposit load:               0
 Latest trade:                  0
 Trades per week:               0
 Avg holding time:              0
Recovery Factor:                0
Long Trades:                    0
Short Trades:                   0
Profit Factor:                  0
Expected Payoff:                0
Average Profit:                 0
Average Loss:                   0
Maximum consecutive losses:     0
Maximal consecutive loss:       0
Monthly growth:                 0
Annual Forecast:               46
Algo trading:                   0
name                            0
last_refresh                    0
dtype: int64

In [58]:
pd.DataFrame([map_label_desc]).to_excel('mapping_col.xlsx',index=False)

In [63]:
engine = sqlalchemy.create_engine('sqlite:///trader_statistics')

In [64]:
engine

Engine(sqlite:///trader_statistics)

In [65]:
df1.to_sql('trader_statistics',engine,index=False,if_exists = 'replace')